<center>
<p><img src="https://mcd.unison.mx/wp-content/themes/awaken/img/logo_mcd.png" width="150">
</p>



<h1>Curso Procesamiento de Lenguaje Natural</h1>

<h3>Transfer learning usando modelos y herramientas de *Hugging Face*</h3>


<p> Julio Waissman Vilanova </p>
<p>
<img src="https://identidadbuho.unison.mx/wp-content/uploads/2019/06/letragrama-cmyk-72.jpg" width="150">
</p>


<a target="_blank" href="https://colab.research.google.com/github/mcd-unison/pln/blob/main/labs/atencion/transfer-hf.ipynb"><img src="https://i.ibb.co/2P3SLwK/colab.png"  style="padding-bottom:5px;"  width="30" /> Ejecuta en Colab</a>

Tomado de [esta libreta](https://colab.research.google.com/github/somosnlp/nlp-de-cero-a-cien/blob/main/4_transformers_aprendizaje_por_transferencia/clasificacion_de_textos.ipynb) pero ajustado para que pueda funcionar con nuevos conjuntos de datos y con los requerimientos necesarios por colab.

</center>


# Clasificación de textos con 🤗 Transformers

> Cómo afinar un modelo de RoBERTa en español para clasificar las reseñas de Amazon.

## Configuración

Si está ejecutando este notebook en Google Colab, ejecute la siguiente celda para instalar las bibliotecas que necesitamos:

In [ ]:
!pip install torch==1.13.1
!pip install transformers datasets
!pip install accelerate

Para compartir tu modelo con la comunidad, primero crea una cuenta en el [Hugging Face Hub](https://huggingface.co/join). A continuación, ejecute la siguiente celda y proporcione su nombre de usuario y contraseña para generar un token de autenticación:

In [ ]:
# Esto sólo funciona en Google Colab! Para los notebooks normales, es necesario ejecutar esto en el terminal
!huggingface-cli login

Si no tienes instalado [Git LFS](https://git-lfs.github.com), puedes hacerlo descomentando y ejecutando la celda de abajo:

In [ ]:
# !apt install git-lfs
# !git config --global user.email "you@example.com"
# !git config --global user.name "Your Name"

## Cargar y explorar los datos

Utilizaremos 🤗 Datasets para cargar y procesar nuestro conjunto de datos. Si no está familiarizado con 🤗 Datasets, vea el siguiente vídeo :)

In [ ]:
from IPython.display import YouTubeVideo

YouTubeVideo("_BZearw7f0w", width=600, height=400)

In [ ]:
from datasets import load_dataset

dataset = load_dataset("tyqiangz/multilingual-sentiments", 'spanish')
dataset

In [ ]:
import random
import pandas as pd
from datasets import ClassLabel
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    "Taken from https://github.com/huggingface/notebooks/blob/master/examples/text_classification.ipynb"

    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

show_random_elements(dataset["train"])

In [ ]:
dataset.set_format("pandas")
df = dataset["train"][:]
df.head()

In [ ]:
df["label"].value_counts()

In [ ]:
df["source"].value_counts()

In [ ]:
dataset.reset_format()

In [ ]:
def formateado(examples):
    return {"labels": examples['label']}

In [ ]:
dataset = dataset.map(formateado)

In [ ]:
show_random_elements(dataset["train"], num_examples=3)

## Tokenizar los textos

In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "BSC-TeMU/roberta-base-bne"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

## Tokenización de textos

Para entender la siguiente sección, vea este breve vídeo sobre la tokenización:

In [ ]:
YouTubeVideo("Yffk5aydLzg", width=600, height=400)

In [ ]:
tokenizer.vocab_size

In [ ]:
text = "¡hola, me llamo Julio Waissman!"
tokenized_text = tokenizer.encode(text)

for token in tokenized_text:
    print(token, tokenizer.decode([token]))

In [ ]:
encoded_text = tokenizer(text, return_tensors="pt")
encoded_text

In [ ]:
def tokenize_texts(examples):
    return tokenizer(examples["text"], truncation=True)

In [ ]:
columns = dataset["train"].column_names
columns.remove("labels")
encoded_dataset = dataset.map(tokenize_texts, batched=True, remove_columns=columns)
encoded_dataset

In [ ]:
encoded_dataset["train"][0]

## Cargar el modelo preentrenado

In [ ]:
from transformers import AutoModelForSequenceClassification

num_labels = 3
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

### De las input IDs a los hidden states

In [ ]:
outputs = model(**encoded_text)
outputs

## Definir las métricas de rendimiento

In [ ]:
from datasets import load_metric

metric = load_metric("accuracy")
metric

In [ ]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

## Afinar el modelo preentrenado

Si quiere entender más profundamente cómo funciona el Entrenador, vea el siguiente vídeo:

In [ ]:
YouTubeVideo("nvBXf7s7vTI", width=600, height=400)

In [ ]:
from transformers import TrainingArguments

model_name = model_checkpoint.split("/")[-1]

batch_size = 16
num_train_epochs=2
num_train_samples = 1_839
train_dataset = encoded_dataset["train"].shuffle(seed=42).select(range(num_train_samples))
logging_steps = len(train_dataset) // (2 * batch_size * num_train_epochs)

training_args = TrainingArguments(
    output_dir="results",
    num_train_epochs=num_train_epochs,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_steps=logging_steps,
    push_to_hub=True,
    push_to_hub_model_id=f"{model_name}-finetuned-multi-sentiment"
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer
)

In [ ]:
trainer.train()

## Empuje hacia el Hugging Face Hub

Para más detalles sobre el envío de modelos al Hub, vea el siguiente vídeo:

In [ ]:
YouTubeVideo("A5IWIxsHLUw", width=600, height=400)

In [ ]:
trainer.push_to_hub()

## Descargue el modelo desde el Hub

In [ ]:
from transformers import pipeline

model_checkpoint = "juliowaissman/roberta-base-bne-finetuned-multi-sentiment"
pipe = pipeline("sentiment-analysis", model=model_checkpoint)

In [ ]:
pipe("Tengo sueño")